In [8]:
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
import pandas as pd
import re

In [9]:
url = 'https://www.comprasal.gob.sv/comprasal_web/resultados'

In [10]:
date_start_xpath = '//*[@id="comprasal_1:fecha_desde_input"]'
date_end_xpath = '//*[@id="comprasal_1:fecha_hasta_input"]'
search_xpath = '//*[@id="comprasal_1:j_idt105"]'
table_xpath = '//*[@id="comprasal_1:principal"]'
next_page_xpath = '/html/body/div[1]/div[2]/div/form/table/tbody/tr[1]/td/div/div[2]/a[3]'
paginator_xpath = '/html/body/div[1]/div[2]/div/form/table/tbody/tr[1]/td/div/div[2]/span'

In [11]:
driver = webdriver.Firefox()
driver.implicitly_wait(3)
driver.get(url)
date_start = driver.find_element_by_xpath(date_start_xpath)
date_end = driver.find_element_by_xpath(date_end_xpath)
search = driver.find_element_by_xpath(search_xpath)
date_start.send_keys('01/01/2020')
date_end.send_keys('30/06/2020')
search.click()

In [12]:
paginator = driver.find_element_by_xpath(paginator_xpath)
paginator.text
limit = int(re.findall('\d+', paginator.text)[-1])
limit

945

In [13]:
tables = []
for i in range(limit):
    while True:
        paginator = driver.find_element_by_xpath(paginator_xpath)
        pos = int(re.findall('\d+', paginator.text)[0])
        if pos == i + 1:
            break
    table = driver.find_element_by_xpath(table_xpath)
    content = table.get_attribute('innerHTML')
    data = pd.read_html(content)
    tables.append(data[0])
    next_page = driver.find_element_by_xpath(next_page_xpath)
    next_page.click()

ImportError: lxml not found, please install it

In [7]:
df = pd.concat(tables)
df.to_csv('compras.csv', index=False)
df.to_excel('compras.xlsx', index=False)